# Final Project Demo Day: The Ultimate NYC Package
# Explanation

New York City can be an overwhelming city to visit. Whether it's your first time or you're a returning visitor, choosing a place to stay and choosing restaurants with so many options can be difficult. Our project will help you choose an AirBnB and restaurants based on the cuisine you would like. Go on and try it!




In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
! pip install folium
import folium

In [16]:
# Read data from csv
airbnb_data_origin= pd.read_csv('NYC_airbnb.csv')
airbnb_data = airbnb_data_origin.filter(['neighbourhood_group', 'price', 'room_type', 'number_of_reviews', 'latitude',
                                         'longitude', 'name', 'host_id'])
restaurants = pd.read_csv('nyc_restaurants.csv')

# Filter by neighborhood
list = ["Manhattan", "Brooklyn", "Queens", "Bronx", "Staten Island"]
while True:
    user_nbhd = input('Choose from Manhattan, Brooklyn, Queens, Bronx, Staten Island or print Quit to quit: ')
    if user_nbhd == "Quit":
        break
    if user_nbhd in list:
        user_nbhd_result = airbnb_data[airbnb_data['neighbourhood_group'] == user_nbhd]
    else:
        print('Please choose from "Manhattan", "Brooklyn", "Queens", "Bronx", "Staten Island" or enter "Quit" to quit')
        continue
# Filter by room type
    list2 = ['Private room', 'Entire home/apt', 'Shared room']
    while True:
        user_room = input("Choose from 'Private room', 'Entire home/apt', 'Shared room'")
        if user_room in list2:
            user_room_result = user_nbhd_result[user_nbhd_result['room_type'] == user_room]
            break
        else:
            print('Please choose from "Private room", "Entire home/apt", "Shared room"')
            continue
    # Filter by price group
    while True:
        user_room_result['price_group'] = np.nan
        user_room_result['price_group'][user_room_result['price'] <= 50] ='0-50'
        user_room_result['price_group'][user_room_result['price'].between(51, 101)] = '51-100'
        user_room_result['price_group'][user_room_result['price'].between(101, 151)] = '101-150'
        user_room_result['price_group'][user_room_result['price'].between(151, 201)] = '151-200'
        user_room_result['price_group'][user_room_result['price'] > 200] ='200+'
        list3 = ['0-50', '51-100', '101-150','151-200','200+']
        user_price = input(f"Choose price range from '0-50', '51-100', '101-150','151-200','200+'")
        if user_price in list3:
            user_price_result = user_room_result[user_room_result['price_group'] == user_price]
            # Sort the rooms by number of reviews from high to low
            final_sort = user_price_result.sort_values(by=['number_of_reviews'], ascending=False)
            CENTER_NYC = (40.7128, -74.0060)
            map = folium.Map(location=CENTER_NYC, zoom_start=12)

            for row in final_sort.head(5).to_records():
                pos = (row['latitude'], row['longitude'])
                marker = folium.Marker(location=pos, popup="%s, ID: %s" % (row['name'], row['host_id'])
                                      )
                map.add_child(marker)
            display(map)
           
            final_choice = final_sort.head(5).filter(['name', 'host_id', 'price'])
            final_df = pd.DataFrame(final_choice)
            display(final_df)
           
            name_choice = input('Please enter the name of the AirBnB you want to stay in: ')
            name = airbnb_data[airbnb_data['name'] == name_choice]
            name_df = pd.DataFrame(name)
           
            map2 = folium.Map(location=CENTER_NYC, zoom_start=12)
            pos2 = (name_df['latitude'], name_df['longitude'])
            marker2 = folium.Marker(location=pos, popup="%s, ID: %s" % (name_df['name'], name_df['host_id']))
            map2.add_child(marker2)
            display(map2)
           
            df = pd.read_csv('nyc_restaurants.csv')
            street_names = df['Street']
            res_names = df['Name']
            res_cuisines = df['Cuisine Description']
            list_of_streets = []
            list_of_restaurants = []
            list_of_cuisines = []
            i = 0
           
            for names in street_names:
                list_of_streets.append(names)
                if (len(list_of_streets) > 150000):
                    break
            for names in res_names:
                list_of_restaurants.append(names)
                if (len(list_of_restaurants) > 150000):
                    break
            for names in res_cuisines:
                list_of_cuisines.append(names)
                if (len(list_of_cuisines) > 150000):
                    break

            def get_user_street():
                user_street = input("Which street or avenue is your AirBnB on? (please enter): ")
                return user_street

            def check_res_in_street(user_street):
                if (user_street) in list_of_streets:
                    return True
                else:
                    return False

            def find_num_of_res(user_street):
                count = 0
                for words in list_of_streets:
                    if (words == user_street):
                        count += 1
                return count

            def ask_show_more(user_street):
                    cuisine_type = input("Enter cuisine: ")
                    food = restaurants[ (restaurants['Street'] == user_street) & (restaurants['Cuisine Description'] == cuisine_type)]
                    value_count = food['Name'].value_counts()
                    count_df = pd.DataFrame(value_count)
                    display(count_df)
                    print("Hope you found the right AirBnB and restaurant of your choice!")

            def main():        
                user_street = get_user_street()

                if (check_res_in_street(user_street) == True):
                    num_res_in_street = find_num_of_res(user_street)
                    if (num_res_in_street > 1):
                        print("There are " + str(num_res_in_street) + " restaurants on this street.")
                        ask_show_more(user_street)
                    else:
                        print("There is 1 restaurant on this street.")
                        ask_show_more(user_street)
                else:
                    print("There are no restaurants on this street. Start over.")
                    main()
           
            main()
           
        else:
            print("Please choose from '0-50', '51-100', '101-150','151-200','200-'")
            continue
           
        break
            # break to exit this while loop    

Choose from Manhattan, Brooklyn, Queens, Bronx, Staten Island or print Quit to quit: Manhattan
Choose from 'Private room', 'Entire home/apt', 'Shared room'Private
Please choose from "Private room", "Entire home/apt", "Shared room"
Choose from 'Private room', 'Entire home/apt', 'Shared room'Private room
Choose price range from '0-50', '51-100', '101-150','151-200','200+'151-200


,name,host_id,price
7884,Spacious Room in East Village/NoHo,7256281,163
18703,Duplex with Private Bathroom by Times Square,92520154,180
1848,ON HIGHLINE! Private room & bath,4323625,179
2267,Summer Special Price Times Square,5944682,185
10055,Private Space with Outdoor Patio,25169596,182


Please enter the name of the AirBnB you want to stay in: Spacious Room in East Village/NoHo


Which street or avenue is your AirBnB on? (please enter): Columbus Avenue
There are 412 restaurants on this street.
Enter cuisine: Chinese


,Name
Canteen 82,25


Hope you found the right AirBnB and restaurant of your choice!
Choose from Manhattan, Brooklyn, Queens, Bronx, Staten Island or print Quit to quit: Quit
